In [1]:
import numpy as np
import json

In [2]:
## Load a json file to following location
with open ('../artifacts/kaggle.json','r') as file:
    data=json.load(file)
data  ##our authentigation

{'username': 'wpnabesinghe', 'key': '86777831f59069f20ceda38b43c88da8'}

In [3]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
  Using cached kaggle-1.6.17.tar.gz (82 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105796 sha256=dc599a5637dd0774148a7ffc55c3c3b16d6e33b1d894279ca533e5449ba02c85
  Stored in directory: c:\users\pasindu\appdata\local\pip\cache\wheels\ff\55\fb\b27a466be754d2a06ffe0e37b248d844f090a63b51becea85d
Successfully built kaggle


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
##
import os
os.environ['KAGGLE_USERNAME']=data['username']
os.environ['KAGGLE_KEY']=data['key']

In [5]:
## Connect with kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api=KaggleApi()
api.authenticate()

In [6]:
##Download the datasets through api
api.dataset_download_files('dineshpiyasamara/sentiment-analysis-dataset',path='../artifacts/')

Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/sentiment-analysis-dataset


In [18]:
## extract zip file
import zipfile
with zipfile.ZipFile('../artifacts/sentiment-analysis-dataset.zip','r') as zip_ref:
    zip_ref.extractall('../artifacts/')

## Model Building

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

data =  pd.read_csv("../artifacts/sentiment_analysis.csv")
data.head(10)


,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


### Data reprocessing

In [10]:
data.duplicated().sum()

0

In [11]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [13]:
## Convert tweet text data to lowercase text
data['tweet'] = data['tweet'].str.lower()
data['tweet'].head(5)

0    #fingerprint #pregnancy test https://goo.gl/h1...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [14]:
import re
url_pattern = r'http\S+|www\S+'

# Remove URLs from the 'tweet' column
data['tweet'] = data['tweet'].str.replace(url_pattern, '', regex=True)
data['tweet'].head()

0    #fingerprint #pregnancy test  #android #apps #...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [23]:
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
import string
# Remove punctuation from the 'tweet' column
data['tweet'] = data['tweet'].str.replace(punctuation_pattern, '', regex=True)
data['tweet'].head()

0    fingerprint pregnancy test  android apps beaut...
1    finally a transparant silicon case  thanks to ...
2    we love this would you go talk makememories un...
3    im wired i know im george i was made that way ...
4    what amazing service apple wont even talk to m...
Name: tweet, dtype: object

In [25]:
data

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beaut...
1,2,0,finally a transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememories un...
3,4,0,im wired i know im george i was made that way ...
4,5,1,what amazing service apple wont even talk to m...
...,...,...,...
7915,7916,0,live out loud lol liveoutloud selfie smile son...
7916,7917,0,we would like to wish you an amazing day make ...
7917,7918,0,helping my lovely 90 year old neighbor with he...
7918,7919,0,finally got my smart pocket wifi stay connecte...


In [26]:
number_pattern = r'\d+'

# Remove numbers from the 'tweet' column
data['tweet'] = data['tweet'].str.replace(number_pattern, '', regex=True)
data['tweet']

0       fingerprint pregnancy test  android apps beaut...
1       finally a transparant silicon case  thanks to ...
2       we love this would you go talk makememories un...
3       im wired i know im george i was made that way ...
4       what amazing service apple wont even talk to m...
                              ...                        
7915    live out loud lol liveoutloud selfie smile son...
7916    we would like to wish you an amazing day make ...
7917    helping my lovely  year old neighbor with her ...
7918    finally got my smart pocket wifi stay connecte...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, Length: 7920, dtype: object

In [30]:
import nltk
from nltk.corpus import stopwords

In [31]:
nltk.download('stopwords',download_dir='../static/model')


[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [33]:
with open ('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

sw


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [35]:
# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in sw]
    return ' '.join(filtered_words)

data['tweet'] = data['tweet'].apply(remove_stopwords)
data['tweet']

0       fingerprint pregnancy test android apps beauti...
1       finally transparant silicon case thanks uncle ...
2       love would go talk makememories unplug relax i...
3       im wired know im george made way iphone cute d...
4       amazing service apple wont even talk question ...
                              ...                        
7915    live loud lol liveoutloud selfie smile sony mu...
7916    would like wish amazing day make every minute ...
7917    helping lovely year old neighbor ipad morning ...
7918    finally got smart pocket wifi stay connected a...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, Length: 7920, dtype: object

In [37]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer


<PorterStemmer>

In [46]:
def preprocess_text(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    return ' '.join(stemmed_words)

# Apply the function to the 'tweet' column
data['tweet'] = data['tweet'].apply(preprocess_text)
data

,id,label,tweet
0,1,0,fingerprint pregnanc test android app beauti c...
1,2,0,final transpar silicon case thank uncl yay son...
2,3,0,love would go talk makememori unplug relax iph...
3,4,0,im wire know im georg made way iphon cute dave...
4,5,1,amaz servic appl wont even talk question unles...
...,...,...,...
7915,7916,0,live loud lol liveoutloud selfi smile soni mus...
7916,7917,0,would like wish amaz day make everi minut coun...
7917,7918,0,help love year old neighbor ipad morn made rea...
7918,7919,0,final got smart pocket wifi stay connect anyti...


### Create Vocabulary

In [47]:
from collections import Counter
vocab=Counter()
vocab

Counter()

In [ ]:
for sentence in data['tweet']:
    vocab.update(sentence.split())



In [49]:
len(vocab)

15155

In [51]:
data.shape  ## Feature = 15155 > records = 7920 , So overfitting

(7920, 3)

In [ ]:
# we select more than 10 times type words
tokens = [key for key in vocab if vocab[key]>10]
tokens

In [55]:
len(tokens)

1143

In [62]:


def save_vocabulary(lines,filename):
    data = '\n'.join(lines)
    file=open(filename,'w',encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(tokens,'../static/model/vocabulary.txt')

#### divide dataset

In [63]:
x=data['tweet']
y=data['label']
x


0       fingerprint pregnanc test android app beauti c...
1       final transpar silicon case thank uncl yay son...
2       love would go talk makememori unplug relax iph...
3       im wire know im georg made way iphon cute dave...
4       amaz servic appl wont even talk question unles...
                              ...                        
7915    live loud lol liveoutloud selfi smile soni mus...
7916    would like wish amaz day make everi minut coun...
7917    help love year old neighbor ipad morn made rea...
7918    final got smart pocket wifi stay connect anyti...
7919    appl barcelona appl store bcn barcelona travel...
Name: tweet, Length: 7920, dtype: object

In [64]:
y

0       0
1       0
2       0
3       0
4       1
       ..
7915    0
7916    0
7917    0
7918    0
7919    0
Name: label, Length: 7920, dtype: int64

In [67]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [68]:
x_train.shape

(6336,)

In [69]:
x_test.shape

(1584,)

### Vectorization

In [77]:
def vectorizer(ds,vocabulary):
    vectorized_lst=[]
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] =1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.float32)
    return vectorized_lst_new

In [78]:
vectorized_x_train=vectorizer(x_train,tokens)
vectorized_x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [79]:
vectorized_x_test=vectorizer(x_test,tokens)
vectorized_x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [80]:
y_train.value_counts()  ## This is a unbalance data sets.I use smort test to balance

label
0    4731
1    1605
Name: count, dtype: int64

In [95]:
## Balance data set
from imblearn.over_sampling import SMOTE
smote=SMOTE()
vectorized_x_train_smote,y_train_smote = smote.fit_resample(vectorized_x_train,y_train)

In [96]:
y_train_smote.value_counts()

label
0    4731
1    4731
Name: count, dtype: int64

## Model Building & Evaluation

In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [92]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def training_scores(y_actu,y_pred):
    acc=round(accuracy_score(y_actu,y_pred),3)
    f1=round(f1_score(y_actu,y_pred),3)
    pr=round(precision_score(y_actu,y_pred),3)
    rec=round(recall_score(y_actu,y_pred),3)
    print(f'training scores:\n\tAccuracy = {acc}\n\tf1_score={f1}\n\tPrecision={pr}\n\t\Recall={rec}')

def validation_scores(y_actu,y_pred):
    acc=round(accuracy_score(y_actu,y_pred),3)
    f1=round(f1_score(y_actu,y_pred),3)
    pr=round(precision_score(y_actu,y_pred),3)
    rec=round(recall_score(y_actu,y_pred),3)
    print(f'training scores:\n\tAccuracy = {acc}\n\tf1_score={f1}\n\tPrecision={pr}\n\t\Recall={rec}')


### Logistic Regression

In [105]:
lr = LogisticRegression()
lr.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = lr.predict(vectorized_x_train_smote)
y_test_pred = lr.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

training scores:
	Accuracy = 0.939
	f1_score=0.94
	Precision=0.914
	\Recall=0.968
training scores:
	Accuracy = 0.869
	f1_score=0.779
	Precision=0.708
	\Recall=0.865


### Decision Tree Regression

In [ ]:
lr = LogisticRegression()
lr.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = lr.predict(vectorized_x_train_smote)
y_test_pred = lr.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

### Naive Bayes

In [107]:
mnb = MultinomialNB()
mnb.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = mnb.predict(vectorized_x_train_smote)
y_test_pred = mnb.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

training scores:
	Accuracy = 0.907
	f1_score=0.911
	Precision=0.868
	\Recall=0.958
training scores:
	Accuracy = 0.872
	f1_score=0.793
	Precision=0.697
	\Recall=0.922


### Decision Tree

In [108]:
dt = DecisionTreeClassifier()
dt.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = dt.predict(vectorized_x_train_smote)
y_test_pred = dt.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

training scores:
	Accuracy = 1.0
	f1_score=1.0
	Precision=1.0
	\Recall=1.0
training scores:
	Accuracy = 0.833
	f1_score=0.682
	Precision=0.689
	\Recall=0.675


### Random forest

In [109]:
rfc = RandomForestClassifier()
rfc.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = rfc.predict(vectorized_x_train_smote)
y_test_pred = rfc.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

training scores:
	Accuracy = 1.0
	f1_score=1.0
	Precision=1.0
	\Recall=1.0
training scores:
	Accuracy = 0.864
	f1_score=0.737
	Precision=0.756
	\Recall=0.72


### Support vector machine

In [110]:
svm = SVC()
svm.fit(vectorized_x_train_smote,y_train_smote)

y_train_pred = svm.predict(vectorized_x_train_smote)
y_test_pred = svm.predict(vectorized_x_test)

training_scores(y_train_smote,y_train_pred)
validation_scores(y_test,y_test_pred)

training scores:
	Accuracy = 0.977
	f1_score=0.977
	Precision=0.96
	\Recall=0.996
training scores:
	Accuracy = 0.878
	f1_score=0.78
	Precision=0.747
	\Recall=0.815


In [111]:
import pickle
with open('../static/model/model.pickle','wb') as file:
    pickle.dump(lr,file)